In [2]:
import os
import sys
import json
import urllib.request
import datetime
import time

In [3]:
client_id = "hysa_dpJm2ciS7VokKF5"
client_secret = "e9D1Xq_F8o"

In [4]:
def getRequestUrl(url):
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)   
    try:
        response = urllib.request.urlopen(request)
        if(response.getcode() == 200):
            print('[%s] Url Request Success'%datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None


In [5]:
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url) #[CODE 1]
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)


In [6]:
def getPostData(post, jsonResult, cnt):    
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    
    pDate = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 
'org_link':org_link,   'link': org_link,   'pDate':pDate})
    return    


In [7]:
def main():
    node = 'news'   # 크롤링 할 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)  #[CODE 2]
    if jsonResponse != None:
        total = jsonResponse['total']
        while ((jsonResponse != None) and (jsonResponse['display'] != 0)):         
            for post in jsonResponse['items']:
                cnt += 1
                getPostData(post, jsonResult, cnt)  #[CODE 3]       
        
            start = jsonResponse['start'] + jsonResponse['display']
            jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]
       
        print('전체 검색 : %d 건' %total)
        with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
            jsonFile = json.dumps(jsonResult,  indent=4, sort_keys=True,  ensure_ascii=False)
                        
            outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('%s_naver_%s.json SAVED' % (srcText, node))

if __name__ == '__main__':  #임포트되지 않고 직접 실행하는 경우라면
    main()                  #main 함수 호출

[2024-07-23 13:02:09.112211] Url Request Success
[2024-07-23 13:02:09.282218] Url Request Success
[2024-07-23 13:02:09.418265] Url Request Success
[2024-07-23 13:02:09.545579] Url Request Success
[2024-07-23 13:02:09.684637] Url Request Success
[2024-07-23 13:02:09.834166] Url Request Success
[2024-07-23 13:02:10.017482] Url Request Success
[2024-07-23 13:02:10.150765] Url Request Success
[2024-07-23 13:02:10.301630] Url Request Success
[2024-07-23 13:02:10.454796] Url Request Success
HTTP Error 400: Bad Request
[2024-07-23 13:02:10.517780] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%ED%91%B8%EB%B0%94%EC%98%A4&start=1001&display=100
전체 검색 : 15892 건
가져온 데이터 : 1000 건
푸바오_naver_news.json SAVED
